🎯 Goal
Now that you have a dds object, we'll:

Run differential expression

Explore results

Make plots:

Volcano plot

PCA plot

In [ ]:
#✅ 1. Run DESeq and extract results
#Since you already have the dds object:

dds <- DESeq(dds)  # Already done — skip if already run
res <- results(dds)
summary(res)

#To get log2 fold changes and p-values:
resOrdered <- res[order(res$padj), ]  # Order by adjusted p-value
head(resOrdered)

In [ ]:
# 2. Volcano plot
#Install the EnhancedVolcano package if not yet installed:


if (!requireNamespace("EnhancedVolcano", quietly = TRUE))
    BiocManager::install("EnhancedVolcano")
library(EnhancedVolcano)

#Then plot:
# Save to PDF
pdf("volcano_plot.pdf", width = 8, height = 6)
EnhancedVolcano(res,
    lab = rownames(res),
    x = "log2FoldChange",
    y = "pvalue",
    pCutoff = 0.05,
    FCcutoff = 1.0,
    title = "Volcano Plot",
    subtitle = "sthh sth")
dev.off()